In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# ================================
# 1. Cargar las bases de entrenamiento, testeo y no respondieron
# ================================

# Completar luego
train = pd.read_csv('(ruta_a_base_entrenamiento.csv)')
test = pd.read_csv('(ruta_a_base_testeo.csv)')
norespondieron = pd.read_csv('(ruta_a_base_norespondieron.csv)')

# ================================
# 2. Definir variables predictoras (X) y variable objetivo (y)
# ================================

# Completar luego
X_train = train[['(columna1)', '(columna2)', '(columna3)']]  # Agregá las variables predictoras
y_train = train['(columna_objetivo)']  # Variable binaria: 1 si desocupado, 0 si no

X_test = test[['(columna1)', '(columna2)', '(columna3)']]
y_test = test['(columna_objetivo)']

X_nsr = norespondieron[['(columna1)', '(columna2)', '(columna3)']]

# ================================
# 3. Regresión logística
# ================================

logit_model = LogisticRegression(max_iter=1000)
logit_model.fit(X_train, y_train)

y_logit_prob = logit_model.predict_proba(X_test)[:,1]
y_logit_pred = (y_logit_prob > 0.5).astype(int)

# ================================
# 4. KNN (K=5)
# ================================

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

y_knn_prob = knn_model.predict_proba(X_test)[:,1]
y_knn_pred = (y_knn_prob > 0.5).astype(int)

# ================================
# 5. Métricas de desempeño
# ================================

def evaluar_modelo(y_true, y_pred, y_prob, nombre_modelo):
    cm = confusion_matrix(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)
    acc = accuracy_score(y_true, y_pred)

    print(f"🔍 {nombre_modelo}")
    print("Matriz de Confusión:\n", cm)
    print(f"AUC: {auc:.4f}")
    print(f"Accuracy: {acc:.4f}\n")

    fpr, tpr, _ = roc_curve(y_true, y_prob)
    plt.plot(fpr, tpr, label=f'{nombre_modelo} (AUC={auc:.2f})')

plt.figure(figsize=(8,6))
evaluar_modelo(y_test, y_logit_pred, y_logit_prob, "Regresión Logística")
evaluar_modelo(y_test, y_knn_pred, y_knn_prob, "KNN (K=5)")

# Gráfico ROC
plt.plot([0,1], [0,1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curvas ROC")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Logit
modelo_final = logit_model
probabilidades_nsr = modelo_final.predict_proba(X_nsr)[:,1]
predicciones_nsr = (probabilidades_nsr > 0.5).astype(int)

# Crear DataFrame con resultados
resultado_nsr = norespondieron.copy()
resultado_nsr['prob_desocupado'] = probabilidades_nsr
resultado_nsr['pred_desocupado'] = predicciones_nsr

# Proporción de desocupados predichos
proporcion_desocupados = np.mean(predicciones_nsr)
print(f"📊 Proporción de desocupados predichos entre los que no respondieron: {proporcion_desocupados:.2%}")